<a href="https://colab.research.google.com/github/ZicoDiegoRR/stable_diffusion_xl_colab_ui/blob/main/beta_modularized_stable_diffusion_xl_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###<font color="black"> » <b><font color="red">Installing Dependencies </b>💿</font> <font color="black"> «
#####ㅤRun this cell first before creating images!

In [ ]:
#@markdown <b>Run this first to install essential libraries!</b><br>
#@markdown <small><p>Required to use the generator.
from IPython.display import clear_output

print("⚙️ | Downloading libraries...")
!git clone https://github.com/ZicoDiegoRR/stable_diffusion_xl_colab_ui.git StableDiffusionXLColabUI
!pip install -r StableDiffusionXLColabUI/requirements.txt
!pip install -r StableDiffusionXLColabUI/requirements_torch.txt

!git clone https://github.com/xinntao/Real-ESRGAN.git RealESRGAN
!pip install -r RealESRGAN/requirements.txt
!pip install facexlib
!pip install gfpgan
!pip install basicsr-fixed
%cd /content/RealESRGAN
!python setup.py develop
%cd /content

clear_output()
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
print("📁 | All essential libraries have been downloaded.")
print("🖌 | You can start generating images now.")

###<font color="black"> »<b><font color="orange">Running Stable Diffusion</b> 🔧</font> <font color="black"> «

In [ ]:
#_______________________________________________________________________________________________________________________________________
#@markdown <b>Run the cell to start!</b>

#@markdown <small>Just run the cell and enjoy. (required to run the cell above first)</small>

#@markdown <small>You can disable Google Drive by not permitting the notebook to access your Google Drive storage.</small>

#@markdown <small>If the runtime got restarted, just run it again.</small>
#_______________________________________________________________________________________________________________________________________

from StableDiffusionXLColabUI.UI.ui_wrapper import UIWrapper
from StableDiffusionXLColabUI.utils import preprocess

# Preprocess the save file, ideas.txt, and GPT-2
cfg, ideas_line, gpt2_pipe = preprocess.run()

# Doing everything
colab_ui = UIWrapper(cfg, ideas_line, gpt2_pipe)


###<font color="black"> » <b><font color="purple">Information </b>✏️📄</font> <font color="black"> «
#####ㅤ
<small>• Text2Img image is saved in Text2Img folder. </small>

<small>• Img2Img image is saved in Img2Img folder. (requires **Reference image** to be filled) </small>

<small>• ControlNet-generated image is saved in ControlNet folder. (requires **Enable Canny**, **Enable Depth Map**, and/or **Enable Open Pose** to be checked, as well as the direct link to the reference image) </small>

<small>• Inpainting-generated image is saved in Inpainting folder. (requires **Inpainting** to be checked, as well as inputting the image and the mask image)</small>

<small> • You can't combine Inpainting and ControlNet.</small>

<small>• IP-Adapter doesn't change the image name.</small>

<small>• You can load LoRAs from your Google Drive by inputting their path. As of now, only LoRAs are supported. </small>

<small>• For ControlNet, leave the image link blank to use the last generated Text2Img image as the reference. Input "inpaint" to use the last generated Inpainting image. And lastly, input "controlnet" to use the last generated ControlNet image. (requires **Enable Canny**, **Enable Depth Map**, **Inpainting**, and/or **Enable Open Pose** to be checked) </small>

***

###<font color="black"> » <b><font color="cyan">Guide </b>🚶🏻📋</font> <font color="black"> «
#####ㅤ

<small> **Prompt:** Basically, this one tells the AI what do you want to see in the image. Sometimes, you have to be strict with your words to align the image with your imagination.

<small> **Model (**checkpoint**):** A saved state during an intense training. This is required to generate the image. The type of model you inputted affects the overall style.

<small> **Model Format:** This is pretty self-explanatory. If you want to use .safetensors model, then set it to "Safe Tensors."

<small> **Steps:** It's simply how many iterations the AI will do in order to generate the image. More doesn't always better. You can look for references online.

<small> **Scale (**Guidance Scale**):** This affects how closely related the image with the prompt. High value can be precise, but low value can add extra uniqueness.

<small> **VAE:** Stands for Variational Autoencoder. It basically controls the color of your image.

<small> **Clip Skip:** Lets the AI skip set amount of layers during generation.

<small> **LoRA:** Stands for Low-Rank Adaptation. It holds weight to be "fed" to the AI. In simple terms, LoRA guides the AI to draw specific characters, style, poses, and so much more. You also need to specify the LoRA's scale, similar to **Scale**.

<small> **Image-to-Image:** This requires exactly one reference image. The AI will turn your image into something different or meaningful.

<small> **Denoising Strength:** Exclusive for Image-to-image, this variable determines how extensive is the denoising process of the reference image. To put it simple, it determines how similar is the generated image compared to the reference. Low value means closely alike to the reference and high value means more changes to the image. Setting this to the maximum value, which is 1, will basically make the AI ignore the reference.

<small> **ControlNet:** Basically a strict instruction based on the inputted image to generate image closely related to the reference.

<small> **Inpainting:** Redrawing an image, but with certain parts of the image changed, just like editing with Photoshop, but AI does the job for you.

<small> **IP-Adapter:** Similar to LoRA, but only follows the inputted image(s). This is stricter than LoRA and sometimes lacks generalization.

<small> **Negative Prompt:** The reverse version of **Prompt**. Instead of telling the AI what do you want, this tells the AI about what do you want to be removed from the image.